<a href="https://colab.research.google.com/github/hasibaasma/Awesome-Bioinformatics/blob/master/validation_hb0039.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

🔹 Step 1 — Install R, Bioconductor, and Python bridges

In [ ]:
# Install R
!sudo apt-get install -y r-base

# Install BiocManager and required R libraries
!R -e "install.packages('BiocManager', repos='https://cloud.r-project.org')"
!R -e "BiocManager::install(c('affy', 'annotate', 'mouse4302.db'))"

# Python R interface
!pip install rpy2

# Load rpy2
%load_ext rpy2.ipython


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
r-base is already the newest version (4.5.2-1.2204.0).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.

R version 4.5.2 (2025-10-31) -- "[Not] Part in a Rumble"
Copyright (C) 2025 The R Foundation for Statistical Computing
Platform: x86_64-pc-linux-gnu

R is free software and comes with ABSOLUTELY NO WARRANTY.
You are welcome to redistribute it under certain conditions.
Type 'license()' or 'licence()' for distribution details.

  Natural language support but running in an English locale

R is a collaborative project with many contributors.
Type 'contributors()' for more information and
'citation()' on how to cite R or R packages in publications.

Type 'demo()' for some demos, 'help()' for on-line help, or
'help.start()' for an HTML browser interface to help.
Type 'q()' to quit R.

> install.packages('BiocManager', repos='https://cloud.r-project.org')
Installing package into 

🔹 Step 2 — Load CEL file and normalize (RMA)

In [ ]:
%%R
library(affy)

read_and_normalize <- function(cel_path) {
    raw <- ReadAffy(filenames = cel_path)
    eset <- rma(raw)
    expr <- exprs(eset)
    df <- data.frame(
        PROBEID = rownames(expr),
        expr = expr[,1]
    )
    return(df)
}


Loading required package: BiocGenerics
Loading required package: generics

Attaching package: ‘generics’

The following objects are masked from ‘package:base’:

    as.difftime, as.factor, as.ordered, intersect, is.element, setdiff,
    setequal, union


Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs

The following objects are masked from ‘package:base’:

    anyDuplicated, aperm, append, as.data.frame, basename, cbind,
    colnames, dirname, do.call, duplicated, eval, evalq, Filter, Find,
    get, grep, grepl, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, saveRDS, table, tapply, unique,
    unsplit, which.max, which.min

Loading required package: Biobase
Welcome to Bioconductor

    Vignettes contain introductory material; view with
    'browseVignettes()'. To cite Bioconductor, see
    'citation("Biobase")', a

🔹 Step 3 — Annotate probes → gene symbols

In [ ]:
%%R
library(mouse4302.db)
library(dplyr)
library(AnnotationDbi)

annotate_to_genes <- function(expr_df) {
    probe2symbol <- AnnotationDbi::select(
        mouse4302.db,
        keys = expr_df$PROBEID,
        keytype = "PROBEID",
        columns = "SYMBOL"
    )

    expr_annot <- expr_df %>% left_join(probe2symbol, by="PROBEID")
    expr_annot <- expr_annot %>% filter(!is.na(SYMBOL))

    expr_collapsed <- expr_annot %>%
        group_by(SYMBOL) %>%
        summarize(expr = mean(expr), .groups="drop")

    colnames(expr_collapsed) <- c("gene", "expr")
    return(expr_collapsed)
}


Loading required package: AnnotationDbi
Loading required package: stats4
Loading required package: IRanges
Loading required package: S4Vectors

Attaching package: ‘S4Vectors’

The following object is masked from ‘package:utils’:

    findMatches

The following objects are masked from ‘package:base’:

    expand.grid, I, unname

Loading required package: org.Mm.eg.db



Attaching package: ‘dplyr’

The following object is masked from ‘package:AnnotationDbi’:

    select

The following objects are masked from ‘package:IRanges’:

    collapse, desc, intersect, setdiff, slice, union

The following objects are masked from ‘package:S4Vectors’:

    first, intersect, rename, setdiff, setequal, union

The following object is masked from ‘package:Biobase’:

    combine

The following objects are masked from ‘package:BiocGenerics’:

    combine, intersect, setdiff, setequal, union

The following object is masked from ‘package:generics’:

    explain

The following objects are masked from ‘package

🔹 Step 4 — Define tissue marker genes

In [ ]:
%%R
markers <- list(
    Brain  = c("Rbfox3","Snap25","Syt1","Tubb3","Gfap","Mbp"),
    Heart  = c("Myh6","Myh7","Tnnt2","Actc1","Myl2","Myl3"),
    Liver  = c("Alb","Apoa1","Fgb","Fga","Fgg","Cyp2e1","Tat"),
    Lung   = c("Sftpa1","Sftpb","Sftpc","Aqp5","Foxj1","Scgb1a1"),
    Kidney = c("Slc34a1","Lrp2","Pvalb","Aqp1","Umod","Pax2"),
    Muscle = c("Acta1","Myh1","Myh2","Myh4","Tnni2","Tnnc2"),
    Skin   = c("Krt14","Krt5","Krt1","Lor","Flg","Col17a1"),
    Blood  = c("Hba-a1","Hba-a2","Hbb-bt","Hbb-bs","Lcp1","Itgam")
)


🔹 Step 5 — Tissue prediction function

In [ ]:
%%R
predict_organ <- function(gene_df) {
    scores <- sapply(names(markers), function(tissue) {
        m <- markers[[tissue]]
        present <- gene_df %>% filter(gene %in% m)
        if(nrow(present)==0) return(0)
        mean(present$expr)
    })
    best <- names(which.max(scores))
    return(list(scores=scores, predicted_organ=best))
}


🔹 Step 6 — Run prediction on your sample

In [ ]:
%%R
cel_file <- "sample_X.CEL.gz"

raw_df <- read_and_normalize(cel_file)
gene_df <- annotate_to_genes(raw_df)
res <- predict_organ(gene_df)

print(res$scores)
print(paste("Predicted organ:", res$predicted_organ))


Background correcting
Normalizing
Calculating Expression
    Brain     Heart     Liver      Lung    Kidney    Muscle      Skin     Blood 
 4.546670 13.440606  6.564637  6.093130  5.774614  5.697959  4.415194 11.000814 
[1] "Predicted organ: Heart"


'getOption("repos")' replaces Bioconductor standard repositories, see
'help("repositories", package = "BiocManager")' for details.
Replacement repositories:
    CRAN: https://cran.rstudio.com
'getOption("repos")' replaces Bioconductor standard repositories, see
'help("repositories", package = "BiocManager")' for details.
Replacement repositories:
    CRAN: https://cran.rstudio.com
trying URL 'https://bioconductor.org/packages/3.22/data/annotation/src/contrib/mouse4302cdf_2.18.0.tar.gz'
Content type 'application/x-gzip' length 3531403 bytes (3.4 MB)
downloaded 3.4 MB


The downloaded source packages are in
	‘/tmp/RtmpbzBBjB/downloaded_packages’

'select()' returned 1:many mapping between keys and columns
In addition: Warning messages:
1: replacing previous import ‘AnnotationDbi::tail’ by ‘utils::tail’ when loading ‘mouse4302cdf’ 
2: replacing previous import ‘AnnotationDbi::head’ by ‘utils::head’ when loading ‘mouse4302cdf’ 
